In [1]:
import os 
import psycopg2
import json #for json files
import sys #used to close json file
import geopandas as gpd

In [ ]:
from functions import insar_jobs
from functions import send_jobs
from functions import process_tifs

In [ ]:
#Connectiong to the postgres database
conn = psycopg2.connect(host='localhost',database = 'postgres', user='postgres',password = 'postgrespw',port=32768)
tableName = "oyster"

In [ ]:
#creating the table for the project
cursor = conn.cursor()
cursor.execute(f"CREATE TABLE IF NOT EXISTS {tableName} (id serial, pair text,site text, insar_name text, reference_date date, vertdisp_path text, coherence_path text)")
conn.commit()
cursor.close()

In [5]:
#create path to the shapefile you want to use
directory = os.getcwd()
inputFiles = os.path.join(directory,"geodata")
shpPath = os.path.join(inputFiles,'BB_Merge.shp')
output = os.path.join(directory, tableName+'_rawData')
if not os.path.exists(output):
    os.makedirs(output)

In [ ]:
jsonFile = open('geodata/Login.json')
login = json.load(jsonFile)

userName = login['UserName']
if userName == "":
    print("Please input Username")
    sys.exit()
password = login['Password']
if password =="":
    print("Please input Password")
    sys.exit()

jsonFile.close()

In [ ]:
#create a list of all insar jobs to be sent to asf
projectJobs = insar_jobs(shpPath,conn,tableName)

In [ ]:
#Send project jobs to ASF 
sendProject = send_jobs(userName,password,projectJobs,output)

In [ ]:
#Crop and reproject all tif files
process = process_tifs(output,tableName,shpPath,conn)

In [9]:
#select insar list between select time frame
afterEnrichment = 52 #number of weeks after enrichment
preEnrichment = 52 #number of weeks before enrichment
shp = gpd.GeoDataFrame.from_file(shpPath)

In [ ]:
print(shp)

In [ ]:
insars = get_insars(tableName,conn,afterEnrichment,preEnrichment,shpPath)

In [ ]:
def get_insars(project,conn,date2,date1):
    date2 = str(d2 + timedelta(weeks = 52))
    date1 = str(d2 - timedelta(weeks = 52))
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT vertdisp_path from {tableName} where reference_date BETWEEN %s AND %s",(date1,date2))
        insars = cursor.fetchall()
        cursor.close()
    except:
        conn = psycopg2.connect(host='localhost',database = 'postgres', user='postgres',password = 'postgrespw',port=32768)